In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.1 MB/s eta 0:00:00


In [3]:
import torch
import timm
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define your class names (adjust based on your dataset)
class_names = ['corn_(maize)_northern_leaf_blight','corn_(maize)_healthy','corn_(maize)_common_rust','corn_(maize)_cercospora_leaf_spot gray_leaf_spot']

# Load the model architecture
model = timm.create_model('vit_base_patch16_224', pretrained=False, num_classes=10)

# Define the path to your model file in Google Drive
model_path = "/content/drive/My Drive/corn_vit_model.pth"  # Adjust the path if needed

# Load the model weights
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# Move model to device (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set model to evaluation mode
model.eval()

print(f"Model loaded successfully from {model_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-3-f8190db9b2df>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))


Model loaded successfully from /content/drive/My Drive/corn_vit_model.pth


In [4]:
# Import necessary libraries
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
from sklearn.metrics import classification_report
from torchvision import transforms

# Define your data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images if needed
    transforms.ToTensor(),
    # You can add more transformations like normalization, augmentation here if needed
])

# Load the full dataset
full_dataset = ImageFolder(root="/content/drive/My Drive/Corn", transform=transform)

# Define the split ratios
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

# Split the dataset into training, validation, and test sets
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

# Create DataLoader for each set
batch_size = 128
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Assuming your model is already loaded and class_names are available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)  # Move model to GPU if available
model.eval()  # Set the model to evaluation mode

# Store all predictions and true labels
all_preds = []
all_labels = []

# Loop through the validation or test data
with torch.no_grad():  # No need to calculate gradients in evaluation mode
    for inputs, labels in val_loader:  # You can change this to `test_loader` for the test set
        inputs, labels = inputs.to(device), labels.to(device)

        # Get predictions
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # Store predictions and true labels
        all_preds.extend(preds.cpu().numpy())  # Convert to CPU and NumPy arrays
        all_labels.extend(labels.cpu().numpy())  # Convert to CPU and NumPy arrays

# Calculate precision, recall, F1 score, and accuracy
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=full_dataset.classes))  # Replace 'class_names' with full_dataset.classes to fetch the classes


Classification Report:
                                                  precision    recall  f1-score   support

corn_(maize)_cercospora_leaf_spot gray_leaf_spot       0.90      0.77      0.83        73
                        corn_(maize)_common_rust       0.98      0.91      0.94       180
                            corn_(maize)_healthy       0.98      1.00      0.99       169
               corn_(maize)_northern_leaf_blight       0.85      0.95      0.90       155

                                        accuracy                           0.93       577
                                       macro avg       0.93      0.91      0.92       577
                                    weighted avg       0.93      0.93      0.93       577



In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Placeholder lists for true and predicted labels
true_labels = []
predicted_labels = []

# Switch model to evaluation mode
model.eval()

# Disable gradient calculation (inference only)
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Get predictions from the model
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # Store the predictions and true labels
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(preds.cpu().numpy())

# Compute accuracy, precision, recall, and f1-score
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Print results
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")


Accuracy: 90.16%
Precision: 91.02%
Recall: 90.16%
F1 Score: 89.73%


In [6]:
import torch
import timm
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
from prettytable import PrettyTable

# Data augmentation (same as in your dataset)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
    transforms.RandomVerticalFlip(),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the full dataset
dataset = ImageFolder(root="/content/drive/My Drive/Corn", transform=transform)

# Splitting the dataset into train, validation, and test sets (65% train, 25% val, 10% test)
train_size = int(0.65 * len(dataset))
val_size = int(0.25 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders
train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True, num_workers=4)
val_loader = DataLoader(dataset=val_dataset, batch_size=128, shuffle=False, num_workers=4)
test_loader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=False, num_workers=4)

# Function to evaluate the model
def evaluate_model(model, data_loader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total  # Return accuracy percentage

# Function to print the table with results
def print_results_table(results):
    table = PrettyTable()
    table.field_names = ["Number of Encoders", "Number of Attention Heads", "Accuracy (%)"]

    for result in results:
        table.add_row([result['encoders'], result['heads'], result['accuracy']])

    print(table)

# Sample model performance evaluation function
def run_model_on_configurations(data_loader, num_classes, device='cuda'):
    # Configuration of encoder depths and attention heads based on common ViT model structures
    configurations = [
        {"encoders": 8, "heads": 8},   # Default ViT settings, you can adjust based on needs
        {"encoders": 8, "heads": 12},
        {"encoders": 12, "heads": 8},
        {"encoders": 12, "heads": 12},
        {"encoders": 16, "heads": 8},
        {"encoders": 16, "heads": 12},
    ]

    results = []

    for config in configurations:
        # Load a ViT model from timm with the specified number of encoder layers and attention heads
        model = timm.create_model(
            'vit_base_patch16_224',
            pretrained=True,
            num_classes=num_classes,
            depth=config['encoders'],  # Number of encoder layers
            num_heads=config['heads']  # Number of attention heads
        )
        model.to(device)

        # Evaluate the model and capture accuracy
        accuracy = evaluate_model(model, data_loader, device)

        # Store results
        results.append({
            "encoders": config['encoders'],
            "heads": config['heads'],
            "accuracy": accuracy
        })

    # Print the results in a table
    print_results_table(results)

# Assuming you have your dataset and DataLoader ready, you can call the function like this:
num_classes = 4  # For the apple dataset, adjust this to match your actual number of classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Run the model on various configurations and print the results
run_model_on_configurations(train_loader, num_classes, device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

+--------------------+---------------------------+--------------------+
| Number of Encoders | Number of Attention Heads |    Accuracy (%)    |
+--------------------+---------------------------+--------------------+
|         8          |             8             | 13.264083100279665 |
|         8          |             12            | 14.702357171394327 |
|         12         |             8             | 29.92409109069117  |
|         12         |             12            | 14.622453056332402 |
|         16         |             8             | 25.56931681981622  |
|         16         |             12            | 19.976028765481423 |
+--------------------+---------------------------+--------------------+


In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
from google.colab import files
import timm

# Define your class names
class_names = ['corn_(maize)_northern_leaf_blight','corn_(maize)_healthy','corn_(maize)_common_rust','corn_(maize)_cercospora_leaf_spot gray_leaf_spot']

# Define the transformations for the image (resize, normalize etc.)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

# Load your model from Google Drive (adjust the model path as needed)
model_path = '/content/drive/MyDrive/apple_vit_model.pth'  # Path to your model file
model = timm.create_model('vit_base_patch16_224', pretrained=False, num_classes=10)  # 4 classes
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()  # Set the model to evaluation mode

# Function to preprocess the image
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")  # Open and ensure 3 channels (RGB)
    image = transform(image).unsqueeze(0)  # Transform and add batch dimension
    return image

# Function to make predictions
def predict(image_path):
    image = preprocess_image(image_path)
    image = image.to("cpu")  # Move image to CPU (since we loaded the model on CPU)

    # Perform prediction
    with torch.no_grad():
        output = model(image)
        _, predicted_class = torch.max(output, 1)

    # Map predicted class to the actual class name
    predicted_label = class_names[predicted_class.item()]
    print(f"Predicted class: {predicted_label}")

# Upload the image from your local system
uploaded = files.upload()

# Get the uploaded file name
image_filename = list(uploaded.keys())[0]

# Call the prediction function on the uploaded image
predict(image_filename)


<ipython-input-8-e2c631e35ac8>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))


Saving Apple-scab-advanced-leaves-Grabowski.jpg to Apple-scab-advanced-leaves-Grabowski.jpg
Predicted class: apple_healthy
